In [9]:
import pandas as pd
import datetime

#Load the game data to pandas data frame
df_season_map  = pd.read_csv('../data/game_dates_season.csv')
df_raw = pd.read_csv('../data/nfl_games_pfr_raw.csv')
df_teams = pd.read_csv('../data/teams.csv')

In [3]:
print(df_raw.head())

   Week  Day        Date    Time           Winner/tie Unnamed: 5  \
0     1  Thu  2024-09-05  8:20PM   Kansas City Chiefs        NaN   
1     1  Fri  2024-09-06  8:15PM  Philadelphia Eagles        NaN   
2     1  Sun  2024-09-08  1:00PM  Pittsburgh Steelers          @   
3     1  Sun  2024-09-08  1:00PM       Houston Texans          @   
4     1  Sun  2024-09-08  1:00PM       Miami Dolphins        NaN   

              Loser/tie Unnamed: 7  PtsW  PtsL  YdsW  TOW  YdsL  TOL  
0      Baltimore Ravens   boxscore    27    20   353    1   452    1  
1     Green Bay Packers   boxscore    34    29   410    3   414    1  
2       Atlanta Falcons   boxscore    18    10   270    0   226    3  
3    Indianapolis Colts   boxscore    29    27   417    0   303    1  
4  Jacksonville Jaguars   boxscore    20    17   400    0   267    1  


In [32]:
#Columns: Week,Day,Date,Time,Winner/tie,Unnamed: 5,Loser/tie,,PtsW,PtsL,YdsW,TOW,YdsL,TOL
df_games = df_raw
df_teams_abbr = df_teams[['Team Name','Team Abbr']]

df_games = pd.merge(df_games, df_season_map, on='Date', how='left')
df_games = pd.merge(df_games, df_teams_abbr, left_on='Winner/tie', right_on='Team Name', how='left')
df_games = pd.merge(df_games, df_teams_abbr, left_on='Loser/tie', right_on='Team Name', how='left')

if len(df_games[df_games['season'].isna()])==0:
    df_games[['season','week']] = df_games[['season','week']].round().astype(int)
    
else:
    print("Some game dates not included in mapping file")
    print(df_games[df_games['season'].isna()])

df_games = df_games.drop(columns=['Unnamed: 7','Unnamed: 0','Team Name_x','Team Name_y']).rename(columns={'Unnamed: 5':'LoserIsHome', 'Winner/tie':'Winner','Loser/tie':'Loser', 'Team Abbr_x':'Winner Abbr','Team Abbr_y':'Loser Abbr'})
df_games['Home Team'] = df_games.apply(lambda row: row['Loser Abbr'] if row['LoserIsHome']=="@" else row['Winner Abbr'], axis=1)
df_games['Away Team'] = df_games.apply(lambda row: row['Winner Abbr'] if row['LoserIsHome']=="@" else row['Loser Abbr'], axis=1)
df_games['Winner Yds'] = df_games.apply(lambda row: row['Winner Abbr'] if row['YdsW']>=row['YdsL'] else row['Loser Abbr'], axis=1)
df_games['Loser Yds'] = df_games.apply(lambda row: row['Winner Abbr'] if row['YdsW']<row['YdsL'] else row['Loser Abbr'], axis=1)
df_games['Margin Yds'] = df_games.apply(lambda row: abs(row['YdsW']-row['YdsL']), axis=1)
df_games['Winner TO'] = df_games.apply(lambda row: row['Winner Abbr'] if row['TOW']>=row['TOL'] else row['Loser Abbr'], axis=1)
df_games['Loser TO'] = df_games.apply(lambda row: row['Winner Abbr'] if row['TOW']<row['TOL'] else row['Loser Abbr'], axis=1)
df_games['Margin TO'] = df_games.apply(lambda row: abs(row['TOW']-row['TOL']), axis=1)

print(df_games.head())

#print(df_games[df_games['Team Abbr_x'].isna()])
### Need to add error handling for unmapped abbreviations

   Week  Day        Date    Time               Winner LoserIsHome  \
0     1  Thu  2024-09-05  8:20PM   Kansas City Chiefs         NaN   
1     1  Fri  2024-09-06  8:15PM  Philadelphia Eagles         NaN   
2     1  Sun  2024-09-08  1:00PM  Pittsburgh Steelers           @   
3     1  Sun  2024-09-08  1:00PM       Houston Texans           @   
4     1  Sun  2024-09-08  1:00PM       Miami Dolphins         NaN   

                  Loser  PtsW  PtsL  YdsW  ...  Winner Abbr  Loser Abbr  \
0      Baltimore Ravens    27    20   353  ...           KC         BAL   
1     Green Bay Packers    34    29   410  ...          PHI          GB   
2       Atlanta Falcons    18    10   270  ...          PIT         ATL   
3    Indianapolis Colts    29    27   417  ...          HOU         IND   
4  Jacksonville Jaguars    20    17   400  ...          MIA         JAC   

   Home Team  Away Team  Winner Yds Loser Yds Margin Yds Winner TO Loser TO  \
0         KC        BAL         BAL        KC         9

In [33]:
df_games.to_csv('../data/nfl_games_pfr.csv')